In [57]:
import snap
from operator import itemgetter
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from random import choice
from pandas import datetools
import datetime
import numpy as np
from __future__ import division

In [58]:
ego = 1

In [59]:
edges = pd.read_csv('data/Ego '+str(ego)+'/ego_edges'+str(ego)+'.csv',',', index_col=None, comment='#')
nodes = pd.read_csv('data/Ego '+str(ego)+'/ego_nodes'+str(ego)+'.csv',',', index_col=None, comment='#')
metadata = pd.read_csv('data/Ego '+str(ego)+'/ego_meta'+str(ego)+'.csv', ',')
metadata['Timestamp']=metadata['Timestamp'].astype('datetime64')
metadata=metadata.sort_values(by=['Timestamp'])

In [60]:
metadata=metadata[metadata['emotion_name'].isin(['Happy', 'Frustrated', 'Lonely', 'Sad', 'Stressed', 
                                                'Disappointed','Annoyed', 'Afraid', 'Angry', 'Lonely', 'Excited', 'Amused',
                                                'Ashamed', 'Anxious', 'Relaxed'])]
        
#metadata=metadata[metadata['category_name'].isin(['Sadness', 'Anger', 'Happiness', 
                                                  #'Fear', 'Surprise', 'Affection'])]


In [61]:
G = nx.from_pandas_edgelist(edges, 'Source', 'Target', True, nx.DiGraph())

In [62]:
i=0
emotions_list = [[0,0]]
for index, vent in metadata.iterrows():
    first_emotion = vent.emotion_id
    next_vents = metadata[(metadata['id']==vent['id']) & 
                         (metadata['Timestamp']>vent['Timestamp'])] 
    next_emotion = next_vents[next_vents.Timestamp == next_vents.Timestamp.min()]['emotion_id']
    
    if next_emotion.values.size==0:
        
        emotions= np.append(first_emotion, 0)
    else:
        
        emotions= np.append(first_emotion, next_emotion)
        
    emotions_list = np.append(emotions_list ,[emotions],axis=0)

In [42]:
pd.DataFrame(emotions_list).to_csv('data/Ego '+str(ego)+'/transition_emotions2.csv',index=None, header=('Source','Target'))

In [43]:
len(emotions_list)

160277

In [44]:
emotions_id = metadata[['emotion_id','emotion_name']].drop_duplicates()
emotions_id = emotions_id.rename(columns={'emotion_id': 'id'})

In [52]:
pd.DataFrame(emotions_id).to_csv('data/Ego '+str(ego)+'/emotions_id2.csv',index=None)

### Markov Transition

In [46]:
emotions_list

array([[  0,   0],
       [541,  26],
       [ 26,  34],
       ...,
       [ 74, 134],
       [134,  32],
       [ 32,   0]])

In [47]:
emotions_list_pd=pd.DataFrame(emotions_list)
emotions_list_pd['count']=1
emotions_list_pd.head()

,0,1,count
0,0,0,1
1,541,26,1
2,26,34,1
3,34,134,1
4,134,26,1


In [48]:
emotions_list_pd = emotions_list_pd.pivot_table(index=[0], columns=[1], aggfunc=sum, fill_value=0)
emotions_list_pd.head()

count                                       ...                           \
1   0    1    5   7   8   9   10  11  13  18  ... 339 461 503  541 563 597   
0                                             ...                            
0     1    0    0   0   0   0   0   0   0   0 ...   0   0   0    0   0   0   
1    25  106   45  25  10  70  39  48  12  12 ...   2   0   0   95   9   0   
5    44   42  179  45  27  45  21  74  14  27 ...   2   1   1  114  17   0   
7    16   31   28  67  17  29  18  37   2  13 ...   2   0   0   78  10   0   
8     8    9   10  14  10  11   9  21   2   1 ...   0   0   0   23   5   0   

                   
1 598 599 600 647  
0                  
0   0   0   0   0  
1   0   9  13   0  
5   0  10  16   0  
7   0   1   6   0  
8   0   2   0   0  

[5 rows x 83 columns]

In [49]:
emotions_list_pd.columns = ['{}'.format(x[1]) for x in emotions_list_pd.columns]
sum_emotions = emotions_list_pd.sum(axis=1)
emotions_list_pd=emotions_list_pd.div(sum_emotions.values,axis='rows')

In [50]:
emotions_list_pd.head()

,0,1,5,7,8,9,10,11,13,18,...,339,461,503,541,563,597,598,599,600,647
0,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
1,0.010865,0.046067,0.019557,0.010865,0.004346,0.030422,0.016949,0.020860,0.005215,0.005215,...,0.000869,0.000000,0.000000,0.041286,0.003911,0.0,0.0,0.003911,0.005650,0.0
5,0.015188,0.014498,0.061788,0.015533,0.009320,0.015533,0.007249,0.025544,0.004833,0.009320,...,0.000690,0.000345,0.000345,0.039351,0.005868,0.0,0.0,0.003452,0.005523,0.0
7,0.010589,0.020516,0.018531,0.044341,0.011251,0.019193,0.011913,0.024487,0.001324,0.008604,...,0.001324,0.000000,0.000000,0.051621,0.006618,0.0,0.0,0.000662,0.003971,0.0
8,0.013378,0.015050,0.016722,0.023411,0.016722,0.018395,0.015050,0.035117,0.003344,0.001672,...,0.000000,0.000000,0.000000,0.038462,0.008361,0.0,0.0,0.003344,0.000000,0.0


In [51]:
pd.DataFrame(emotions_list_pd).to_csv('data/Ego '+str(ego)+'/markov_emotions2.csv')